In [1]:
import pandas as pd

In [2]:
# Read raw data
df = pd.read_csv('data/2022/flights_combined.csv')

C:\Users\peter\AppData\Local\Temp\ipykernel_13804\302095532.py:2: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/2022/flights_combined.csv')


In [3]:
# Replace missing delays ith 0 as these not delayed
df['arrival.delay'].fillna(0, inplace=True)

In [4]:
# Extract year from scheduled datetime
df['year'] = pd.DatetimeIndex(df['departure.scheduledTime']).year

In [5]:
# Extract month from scheduled datetime
df['month'] = pd.DatetimeIndex(df['departure.scheduledTime']).month

In [6]:
# Extract day from scheduled datetime
df['day'] = pd.DatetimeIndex(df['departure.scheduledTime']).day

In [7]:
# Convert to date only
df['date'] = pd.to_datetime(df['departure.scheduledTime'])
# Extract weekday from date
df['weekday'] = df['date'].dt.dayofweek

In [8]:
#TODO: get scheduled time as feature
#TODO: get sheduled arrival times and find diff between actual and maybe use as target for regression

# Extract time from scheduled datetime
df['time'] = pd.DatetimeIndex(df['departure.scheduledTime']).time

In [9]:
# Read in airport location data
df_airports = pd.read_csv('data/GlobalAirportDatabase.csv')
# Drop unwanted columns
df_airports.drop(['latitude_degrees', 'latitude_minutes', 'latitude_seconds',
                  'latitude_direction', 'longitude_degrees', 'longitude_minutes',
                  'longitude_seconds', 'longitude_direction'], axis=1, inplace=True)

In [10]:
df_airports.head()

,ICAO,IATA,airport_name,city,country,altitude,latitude_decimal_degrees,longitude_decimal_degrees
0,ayga,gka,goroka,goroka,papua new guinea,1610,-6.082,145.392
1,ayla,lae,NaN,lae,papua new guinea,0,0.000,0.000
2,aymd,mag,madang,madang,papua new guinea,7,-5.207,145.789
3,aymh,hgu,mount hagen,mount hagen,papua new guinea,1643,-5.826,144.296
4,aynz,lae,nadzab,nadzab,papua new guinea,73,-6.570,146.726


In [11]:
# Merge in airport location data for departure airports
df2 = pd.merge(df, df_airports, left_on='departure.iataCode', right_on='IATA', how='left')
# Rename to flag these as departure
df2 = df2.rename({"latitude_decimal_degrees": "latitude_departure", "longitude_decimal_degrees": "longitude_departure"}, axis=1)

In [12]:
# Merge in airport location data for arrival airports
df2 = pd.merge(df2, df_airports, left_on='arrival.iataCode', right_on='IATA', how='left')
# Rename to flag these as arrival
df2 = df2.rename({"latitude_decimal_degrees": "latitude_arrival", "longitude_decimal_degrees": "longitude_arrival"}, axis=1)

In [13]:
# Raname to identify departure and arrival columns
df2.columns = df2.columns.str.replace("_x", "_departure")
df2.columns = df2.columns.str.replace("_y", "_arrival")

In [14]:
df2.head()

,airline.name,departure.terminal,departure.gate,departure.scheduledTime,departure.actualTime,arrival.scheduledTime,arrival.actualTime,status,departure.iataCode,arrival.delay,...,latitude_departure,longitude_departure,ICAO_arrival,IATA_arrival,airport_name_arrival,city_arrival,country_arrival,altitude_arrival,latitude_arrival,longitude_arrival
0,klm,NaN,NaN,2022-01-05t08:35:00.000,NaN,2022-01-05t09:50:00.000,NaN,active,koi,0.0,...,58.958,-2.905,egph,edi,edinburgh,edinburgh,uk,42.0,55.950,-3.373
1,klm,NaN,NaN,2022-01-06t08:35:00.000,2022-01-06t08:34:00.000,2022-01-06t09:50:00.000,NaN,active,koi,0.0,...,58.958,-2.905,egph,edi,edinburgh,edinburgh,uk,42.0,55.950,-3.373
2,klm,NaN,NaN,2022-01-07t08:35:00.000,2022-01-07t08:47:00.000,2022-01-07t09:50:00.000,NaN,active,koi,0.0,...,58.958,-2.905,egph,edi,edinburgh,edinburgh,uk,42.0,55.950,-3.373
3,klm,NaN,NaN,2022-01-07t19:00:00.000,2022-01-07t18:44:00.000,2022-01-07t19:50:00.000,NaN,active,koi,0.0,...,58.958,-2.905,egpd,abz,dyce,aberdeen,uk,66.0,57.204,-2.200
4,klm,NaN,NaN,2022-01-08t10:05:00.000,2022-01-08t10:25:00.000,2022-01-08t11:20:00.000,NaN,active,koi,3.0,...,58.958,-2.905,egph,edi,edinburgh,edinburgh,uk,42.0,55.950,-3.373


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296080 entries, 0 to 296079
Data columns (total 33 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   airline.name             296080 non-null  object        
 1   departure.terminal       168407 non-null  object        
 2   departure.gate           232927 non-null  object        
 3   departure.scheduledTime  296080 non-null  object        
 4   departure.actualTime     185879 non-null  object        
 5   arrival.scheduledTime    296080 non-null  object        
 6   arrival.actualTime       4117 non-null    object        
 7   status                   296080 non-null  object        
 8   departure.iataCode       296080 non-null  object        
 9   arrival.delay            296080 non-null  float64       
 10  arrival.iataCode         296080 non-null  object        
 11  year                     296080 non-null  int64         
 12  month           

In [18]:
df2['delay_calc'] = pd.DatetimeIndex(df2['arrival.scheduledTime']).time - pd.DatetimeIndex(df2['arrival.actualTime']).time

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'NaTType'

In [20]:
df2['three_hr'] = df2['arrival.delay'] >= 180

In [25]:
len(df2)

296080

In [27]:
df2.loc[df2['arrival.delay']>50, :]

,airline.name,departure.terminal,departure.gate,departure.scheduledTime,departure.actualTime,arrival.scheduledTime,arrival.actualTime,status,departure.iataCode,arrival.delay,...,longitude_departure,ICAO_arrival,IATA_arrival,airport_name_arrival,city_arrival,country_arrival,altitude_arrival,latitude_arrival,longitude_arrival,three_hr
4735,klm,NaN,3.0,2022-11-25t18:00:00.000,2022-11-25t18:53:00.000,2022-11-25t19:05:00.000,NaN,active,abz,52.0,...,-2.200,egpb,lsi,sumburgh,sumburgh,uk,7.0,59.879,-1.296,False
8790,jet2,NaN,NaN,2022-02-14t09:00:00.000,2022-02-14t09:11:00.000,2022-02-14t10:45:00.000,NaN,active,bhx,102.0,...,-1.748,lirf,fco,fiumicino,rome,italy,4.0,41.813,12.253,False
10344,ryanair,NaN,NaN,2022-04-01t15:55:00.000,2022-04-01t16:10:00.000,2022-04-01t13:20:00.000,NaN,active,bhx,213.0,...,-1.748,eidw,dub,dublin,dublin,ireland,74.0,53.421,-6.270,True
24504,virgin atlantic,NaN,NaN,2022-11-15t10:00:00.000,2022-11-15t10:41:00.000,2022-11-15t19:36:00.000,NaN,active,bhx,91.0,...,-1.748,otbd,doh,doha international,doha,qatar,11.0,25.261,51.565,False
31268,jet2,NaN,NaN,2022-07-31t06:35:00.000,2022-07-31t07:27:00.000,2022-07-31t11:35:00.000,NaN,active,brs,51.0,...,-2.719,lgkr,cfu,ioannis kapodistrias international,kerkyra/corfu,greece,2.0,39.602,19.912,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264733,ryanair,NaN,NaN,2022-07-04t08:20:00.000,2022-07-04t09:04:00.000,2022-07-04t09:50:00.000,NaN,active,stn,149.0,...,0.235,leal,alc,alicante,alicante,spain,44.0,38.282,-0.558,False
287888,virgin atlantic,NaN,NaN,2022-10-02t22:15:00.000,2022-10-02t23:03:00.000,2022-10-03t00:12:00.000,NaN,active,stn,136.0,...,0.235,kmsp,msp,minneapolis st paul international,minneapolis,usa,257.0,44.880,-93.217,False
288395,ryanair,NaN,84.0,2022-11-04t10:05:00.000,2022-11-04t10:51:00.000,2022-11-04t13:50:00.000,NaN,active,stn,55.0,...,0.235,lpfu,fnc,NaN,funchal,madeira,0.0,0.000,0.000,False
288476,ryanair,NaN,81.0,2022-11-04t18:05:00.000,2022-11-04t19:00:00.000,2022-11-04t21:35:00.000,NaN,active,stn,51.0,...,0.235,lemd,mad,barajas,madrid,spain,610.0,40.472,-3.561,False


In [258]:
# TODO: Calculate distance between cities with GPS long lat data.

,airline.name,departure.terminal,departure.gate,departure.scheduledTime,departure.actualTime,arrival.scheduledTime,arrival.actualTime,status,departure.iataCode,arrival.delay,...,latitude_departure,longitude_departure,ICAO_arrival,IATA_arrival,airport_name_arrival,city_arrival,country_arrival,altitude_arrival,latitude_arrival,longitude_arrival
0,klm,NaN,NaN,2022-01-05t08:35:00.000,NaN,2022-01-05t09:50:00.000,NaN,active,koi,0.0,...,58.958,-2.905,egph,edi,edinburgh,edinburgh,uk,42.0,55.950,-3.373
1,klm,NaN,NaN,2022-01-06t08:35:00.000,2022-01-06t08:34:00.000,2022-01-06t09:50:00.000,NaN,active,koi,0.0,...,58.958,-2.905,egph,edi,edinburgh,edinburgh,uk,42.0,55.950,-3.373
2,klm,NaN,NaN,2022-01-07t08:35:00.000,2022-01-07t08:47:00.000,2022-01-07t09:50:00.000,NaN,active,koi,0.0,...,58.958,-2.905,egph,edi,edinburgh,edinburgh,uk,42.0,55.950,-3.373
3,klm,NaN,NaN,2022-01-07t19:00:00.000,2022-01-07t18:44:00.000,2022-01-07t19:50:00.000,NaN,active,koi,0.0,...,58.958,-2.905,egpd,abz,dyce,aberdeen,uk,66.0,57.204,-2.200
4,klm,NaN,NaN,2022-01-08t10:05:00.000,2022-01-08t10:25:00.000,2022-01-08t11:20:00.000,NaN,active,koi,3.0,...,58.958,-2.905,egph,edi,edinburgh,edinburgh,uk,42.0,55.950,-3.373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296075,jet2,1.0,13.0,2022-12-20t09:45:00.000,NaN,2022-12-20t14:25:00.000,NaN,active,stn,0.0,...,51.885,0.235,gcts,tfs,tenerife sur,tenerife,canary islands,64.0,28.044,-16.572
296076,jet2,1.0,3.0,2022-12-20t10:30:00.000,NaN,2022-12-20t14:50:00.000,NaN,active,stn,0.0,...,51.885,0.235,gclp,lpa,gran canaria,gran canaria,canary islands,24.0,27.932,-15.386
296077,jet2,1.0,19.0,2022-12-20t12:25:00.000,NaN,2022-12-20t16:35:00.000,NaN,active,stn,0.0,...,51.885,0.235,gcrr,ace,lanzarote,lanzerote,canary islands,15.0,28.945,-13.605
296078,jet2,1.0,19.0,2022-12-20t14:15:00.000,NaN,2022-12-20t18:55:00.000,NaN,active,stn,0.0,...,51.885,0.235,gcts,tfs,tenerife sur,tenerife,canary islands,64.0,28.044,-16.572
